## Практическая работа

## Цели практической работы

*  решить реальную задачу;
*  потренироваться в обработке данных;
*  обучить различные модели классификации, подобрать гиперпараметры и выбрать лучшую модель;
*  добиться наилучшего качества в задаче.

## Что входит в практическую работу

*  исследование датасета и обработка данных (работа с пропущенными и ошибочными значениями);
*  обучение различных моделей классификации с параметрами по умолчанию;
*  подбор гиперпараметров моделей;
*  смешивание моделей;
*  оценка качества моделей.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания: впишите свой код (или, если требуется, текст) в ячейки после комментариев. 

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

### 1. Загрузите тренировочные и тестовые датасеты

In [91]:
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

Xtrain = pd.read_csv("TrainData.csv")
Xtest = pd.read_csv("TestData.csv")

### 2. Изучите тренировочные и тестовые данные на наличие:
- пропусков,
- ошибочных значений.

Обработайте пропуски и ошибочные значения способом, выбранным по своему усмотрению.

In [92]:
Xtrain.shape

(7500, 15)

In [93]:
# процент пропущенных значений
def missing(df):
    n = len(df)
    missing_values = {}
    for i in df.columns:
        missing_values[i] = len(df[df[i].isna()]) / n

    return sorted(missing_values.items(), key = lambda x: x[1], reverse=True)
    
missing(Xtrain)

[('f7', 0.25),
 ('f2', 0.01),
 ('f11', 0.0013333333333333333),
 ('f1', 0.0),
 ('f3', 0.0),
 ('f4', 0.0),
 ('f5', 0.0),
 ('f6', 0.0),
 ('f8', 0.0),
 ('f9', 0.0),
 ('f10', 0.0),
 ('f12', 0.0),
 ('f13', 0.0),
 ('f14', 0.0),
 ('target', 0.0)]

In [94]:
Xtrain[['f7', 'f2', 'f11']].describe()

,f7,f2,f11
count,5625.000000,7425.000000,7490.000000
mean,0.240556,238.905308,1068.693992
std,0.147371,76.198282,7430.344770
min,0.000000,64.259070,0.000000
25%,0.115721,218.593930,0.000000
50%,0.241573,218.593930,0.000000
75%,0.439103,218.593930,0.000000
max,0.482222,575.396825,99999.000000


In [95]:
# f7 заполним средним значением
Xtrain['f7'] = Xtrain['f7'].fillna(Xtrain['f7'].mean())

In [96]:
# f2, f11 заполним самым распространенным значением
Xtrain['f2'] = Xtrain['f2'].fillna(Xtrain['f2'].mode()[0])
Xtrain['f11'] = Xtrain['f11'].fillna(Xtrain['f11'].mode()[0])

In [97]:
Xtest.shape

(2500, 15)

In [98]:
missing(Xtest)

[('f1', 0.0),
 ('f2', 0.0),
 ('f3', 0.0),
 ('f4', 0.0),
 ('f5', 0.0),
 ('f6', 0.0),
 ('f7', 0.0),
 ('f8', 0.0),
 ('f9', 0.0),
 ('f10', 0.0),
 ('f11', 0.0),
 ('f12', 0.0),
 ('f13', 0.0),
 ('f14', 0.0),
 ('target', 0.0)]

In [99]:
# нормализуем данные
sc = StandardScaler()
Xtrain.iloc[:, :-1] = pd.DataFrame(sc.fit_transform(Xtrain.iloc[:, :-1]), columns=Xtrain.iloc[:, :-1].columns)
Xtest.iloc[:, :-1] = pd.DataFrame(sc.fit_transform(Xtest.iloc[:, :-1]), columns=Xtest.iloc[:, :-1].columns)

### 3. Оцените баланс классов в задаче
- Затем попытайтесь устно ответить на вопрос, можно ли использовать accuracy как метрику качества в задаче? 

In [100]:
Xtrain['target'].value_counts()

0    5708
1    1792
Name: target, dtype: int64

Обучающая выборка несбалансирована, поэтому метрика accuracy будет недостаточно точно определять качество модели.

### 3. Постройте baseline-модель:
- разбейте TrainData на тренировочные (Train) и тестовые данные (Test); 
- обучите KNN, LogisticRegression и SVC с параметрами по умолчанию на тренировочных данных (Train);
- примените модели на тестовых данных (Test) и вычислите значение метрики f1.

In [101]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

X = Xtrain.drop(['target'], axis=1)
y = Xtrain['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier()
lr = LogisticRegression()
svc = SVC()

knn.fit(X_train, y_train)
lr.fit(X_train, y_train)
svc.fit(X_train, y_train)

SVC()

In [102]:
knn_pred = knn.predict(X_test)
lr_pred = lr.predict(X_test)
svc_pred = svc.predict(X_test)

In [103]:
print('knn:', f1_score(y_test, knn_pred))
print('lr:', f1_score(y_test, lr_pred))
print('svc:', f1_score(y_test, svc_pred))

knn: 0.6081694402420575
lr: 0.6188197767145136
svc: 0.6127946127946128


### 4. Улучшите модели
Попробуйте улучшить качество обученных моделей:
- можете подбирать гиперпараметры моделей (лучше это делать по кросс-валидации на Train, то есть с помощью использования GridSearchCV на Train);
- можете задавать class_weights;
- можете вручную или при помощи методов Python генерировать новые признаки и/или удалять существующие.

Это самая важная и творческая часть задания. Проводите как можно больше экспериментов!

Проведите минимиум три эксперимента: для каждого типа модели минимум один эксперимент.

In [116]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors': [i for i in range(1, 20, 2)], 
              'weights': ['uniform', 'distance']}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters, scoring='f1', cv=3)
clf.fit(X_train, y_train)
knn_bparams = clf.best_params_
knn_bparams

{'n_neighbors': 13, 'weights': 'distance'}

In [117]:
knn = KNeighborsClassifier(**knn_bparams)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
knn_score = f1_score(y_test, knn_pred)
print('knn:', knn_score)

knn: 0.6003110419906689


In [118]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=UserWarning)

parameters = {'class_weight': ['balanced', None],
             'C': [i / 5 for i in range(1, 10)]}
lr = LogisticRegression()
clf = GridSearchCV(lr, parameters, scoring='f1', cv=3)
clf.fit(X_train, y_train)
lr_bparams = clf.best_params_
lr_bparams

{'C': 1.6, 'class_weight': 'balanced'}

In [119]:
lr = LogisticRegression(**lr_bparams)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_score = f1_score(y_test, lr_pred)
print('lr:', lr_score)

lr: 0.6527472527472528


In [120]:
parameters = {'C': [i / 2 for i in range(1, 4)], 
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'class_weight': ['balanced']}
svc = SVC()
clf = GridSearchCV(svc, parameters, scoring='f1', cv=3)
clf.fit(X_train, y_train)
svc_bparams = clf.best_params_
svc_bparams

{'C': 1.5, 'class_weight': 'balanced', 'kernel': 'rbf'}

In [127]:
svc = SVC(**svc_bparams, probability=True)
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_test)
svc_score = f1_score(y_test, svc_pred)
print('svc:', svc_score)

svc: 0.6524520255863541


### 5. Оцените на отложенной выборке качество наилучшей модели
В пунктах 3 и 4 вы построили много разных моделей.

Возьмите ту, которая дала наилучшее качество на тестовых данных (Test). Примените её на отложенной выборке (TestData) и выведите на экран значение метрики f1.

In [128]:
best_score = max(knn_score, lr_score, svc_score)
if knn_score == best_score:
    print('knn:', f1_score(Xtest['target'], knn.predict(Xtest.iloc[:, :-1])))
elif lr_score == best_score:
    print('lr:', f1_score(Xtest['target'], lr.predict(Xtest.iloc[:, :-1])))
else:
    print('svc:', f1_score(Xtest['target'], svc.predict(Xtest.iloc[:, :-1])))

lr: 0.5216494845360825


### 6. Выполните хитрый трюк
Часто смешивание различных моделей даёт улучшение итогового предсказания. Попробуйте смешать две лучшие модели по формуле:
$$pred_{final} = \alpha\cdot pred_1 + (1-\alpha)\cdot pred_2$$.

Значение $\alpha$ подберите в цикле по Test-выборке. Оцените качество на отложенной выборке.

Удалось ли добиться улучшения качества?

In [135]:
def predict_fin(pred1, pred2, alpha):
    return [1 if alpha * pred1[i][1] + (1 - alpha) * pred2[i][1] > 0.5 else 0 for i in range(len(pred1))]


lr_proba = lr.predict_proba(X_test)
svc_proba = svc.predict_proba(X_test)
best_alpha = 0
best_score = 0

for i in range(0, 11):
    cur_score = f1_score(y_test, predict_fin(lr_proba, svc_proba, i / 10))
    if cur_score > best_score:
        best_score = cur_score
        best_alpha = i / 10
        
best_score, best_alpha

(0.6674907292954264, 0.4)

In [136]:
lr_proba = lr.predict_proba(Xtest.iloc[:, :-1])
svc_proba = svc.predict_proba(Xtest.iloc[:, :-1])
f1_score(Xtest['target'], predict_fin(lr_proba, svc_proba, best_alpha))

0.3764705882352941

### 7. Сделайте выводы

Запишите в отдельной ячейке текстом выводы о проделанной работе. Для этого ответьте на вопросы:
- Какие подходы вы использовали для улучшения работы baseline-моделей?
- Какого максимального качества удалось добиться на Test-данных?
- Какое при этом получилось качество на отложенной выборке? 
- Ваша модель переобучилась, недообучилась или обучилась как надо?

In [112]:
# Для улучшения качества моделей были подобраны параметры с помощью кросс-валидации, две лучшие модели смешали
# На тестовых данных удалось добиться точности f1_score = 0.6674907292954264
# На отложенной выборке точность уменьшилась до 0.3764705882352941 из чего можно сделать вывод, что модель переобучилась

Важный комментарий! В реальных задачах не следует ожидать, что машинным обучением всегда удастся решить задачу с хорошим качеством. Но использовать все имеющиеся у вас в арсенале методы для достижения наилучшего результата нужно.